In [ ]:
# This script tries to approximate the cost of the DTLS handshake

import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math
import numpy as np
from scipy import optimize
import sys
from sklearn.metrics import r2_score

figsize=(4.5, 3)

TUMBlue='#3070b3'
TUMGreen='#a2ad00'
TUMOrange='#e37222'

In [ ]:
# Read the data
ssl = pd.read_csv("../data-files/ecdhe-openssl.csv")
for _,r in ssl.iterrows():
    r['openssl'] /= r['numConns']

In [ ]:
#print(ssl['openssl'])

# This function will be fitted to the data
def fitfunc(x,t,o):
    # This is the actual mathematical function
    def runner(a):
        return (t/a) + o
    # Magic in case curve_fit hands us an array
    if isinstance(x, np.ndarray):
        ret = []
        for s in x:
            ret += [runner(s)]
        return ret
    else:
        return runner(x)

#params = [1.02,2**11,600]
# Try to fit the data
params, _ = optimize.curve_fit(fitfunc, ssl['numConns'], ssl['openssl'])

print(params)

fit = []
for s in ssl['numConns']:
    fit += [fitfunc(s,params[0],params[1])]
    
    
r2 = r2_score(ssl['openssl'], fit) 
print("R2 score: " + str(r2))
    
fig = plt.figure(figsize=figsize, dpi=300)

plt.ylabel('Cycles per Connection')
plt.xlabel('\#Connection')

plt.ylim([0,8500000])

plt.plot(ssl['numConns'], ssl['openssl'], label="data", color=TUMBlue)
plt.plot(ssl['numConns'], fit, label="fit", color=TUMOrange)
plt.legend()
plt.grid(b=True)
#plt.savefig('../pdf-output/ecdhe-openssl.pdf',bbox_inches='tight')
#plt.show()

from matplotlib2tikz import save as tikz_save
tikz_save('../tex-output/ecdhe-openssl.tex',
          figureheight='3.5cm',
          figurewidth='.9\\columnwidth',
          extra_axis_parameters = {"mark options={scale=.15}",
                                   "legend pos = north east"}
         )